# Introduction

This notebook prepares the stellar catalog for the [Pocket Finder-Chart Atlas](https://github.com/alanwatsonforster/pocket-finder-chart-atlas).

# Instructions

1. Run this code.

2. Download stars.fits.

# Prolog

In [1]:
import os

import math

import numpy as np

from astropy.table import Table

We use the Tycho-2 catalog. This is 99% complete to V_T = 11.0, which is deep enough for our needs.

An alternative would be to use the Gaia catalog. However, this is much deeper, and hence larger, than we need.

Download the catalog data from [CDS](https://cdsarc.cds.unistra.fr/viz-bin/cat/I/259#/browse) and unpack it.

In [2]:
!test -f l_259.tar.gz || curl -o l_259.tar.gz https://cdsarc.cds.unistra.fr/viz-bin/nph-Cat/tar.gz?I/259
!test -f tyc2.dat.00.gz || tar -xzf l_259.tar.gz
!test -f tyc2.dat || gunzip -c tyc2.dat.*.gz >tyc2.dat
!test -f suppl_1.dat || gunzip -c suppl_1.dat.gz >suppl_1.dat
!test -f suppl_2.dat || gunzip -c suppl_2.dat.gz >suppl_2.dat
!ls -l *.dat

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  160M    0  160M    0     0  24.6M      0 --:--:--  0:00:06 --:--:-- 26.0M
-rw-r--r-- 1 root root   2163324 Mar 21 23:36 suppl_1.dat
-rw-r--r-- 1 root root    132936 Mar 21 23:36 suppl_2.dat
-rw-r--r-- 1 root root 525761991 Mar 21 23:36 tyc2.dat


Convert the ASCII files into FITS tables.

In [3]:
if not os.path.exists("tyc2.fits"):
  t = Table.read("tyc2.dat", readme="ReadMe", format="ascii.cds")
  t.write("tyc2.fits")
if not os.path.exists("suppl_1.fits"):
  t = Table.read("suppl_1.dat", readme="ReadMe", format="ascii.cds")
  t.write("suppl_1.fits")
if not os.path.exists("suppl_2.fits"):
  t = Table.read("suppl_2.dat", readme="ReadMe", format="ascii.cds")
  t.write("suppl_2.fits")
!ls -l *.fits

-rw-r--r-- 1 root root   2476800 Mar 21 23:38 suppl_1.fits
-rw-r--r-- 1 root root    175680 Mar 21 23:38 suppl_2.fits
-rw-r--r-- 1 root root 645157440 Mar 21 23:38 tyc2.fits


In [4]:
tyc2 = Table.read("tyc2.fits")
tyc2s1 = Table.read("suppl_1.fits")
print(tyc2.info)
print(tyc2s1.info)

<Table length=2539913>
   name    dtype     unit                    description                    class      n_bad 
--------- ------- ---------- ------------------------------------------- ------------ -------
     TYC1   int64                    [1,9537]+= TYC1 from TYC or GSC (1) MaskedColumn       0
     TYC2   int64                    [1,12121]  TYC2 from TYC or GSC (1) MaskedColumn       0
     TYC3   int64                           [1,3]      TYC3 from TYC (1) MaskedColumn       0
    pflag  bytes1                           [ PX] mean position flag (2) MaskedColumn 2417370
   RAmdeg float64        deg       Mean Right Asc, ICRS, epoch=J2000 (3) MaskedColumn  109445
   DEmdeg float64        deg         Mean Decl, ICRS, at epoch=J2000 (3) MaskedColumn  109445
     pmRA float64   mas / yr           Proper motion in RA*cos(dec) (12) MaskedColumn  109445
     pmDE float64   mas / yr                   Proper motion in Dec (12) MaskedColumn  109445
 e_RAmdeg   int64        mas         

Select stars brighter than magnitude 10. We do another cut latter, but this is a quick optimization.

TODO: Add proper motions. Convert from VT to V.

In [5]:
mask = (tyc2["VTmag"] <= 10)
t = tyc2[mask]
TYC = list(tyc1 + "-" + tyc2 + "-" + tyc3 for tyc1, tyc2, tyc3 in zip(t["TYC1"].astype(str), t["TYC2"].astype(str), t["TYC3"].astype(str)))
stars = Table([TYC, t["HIP"], t["RAdeg"], t["DEdeg"], t["VTmag"]], 
              names=["TYC", "HIP", "alpha", "delta", "mag"],
              descriptions=["TYC Identifier", "HIP Identifier", "Right Ascension (J2000)", "Declination (J2000)", "VT Magnitude"])

print(stars.info)

<Table length=327931>
 name  dtype  unit       description          class     n_bad 
----- ------- ---- ----------------------- ------------ ------
  TYC   str12               TYC Identifier       Column      0
  HIP   int64               HIP Identifier MaskedColumn 219796
alpha float64  deg Right Ascension (J2000)       Column      0
delta float64  deg     Declination (J2000)       Column      0
  mag float64  mag            VT Magnitude MaskedColumn      0



Add stars from Supplement 1 at or brighter than the limiting magnitude.

TODO: Add proper motions. Convert from VT or Hp to V.

In [6]:
mask = (tyc2s1["VTmag"] <= 10)
ts1 = tyc2s1[mask]
TYC = list((tyc1 + "-" + tyc2 + "-" + tyc3) for tyc1, tyc2, tyc3 in zip(ts1["TYC1"].astype(str), ts1["TYC2"].astype(str), ts1["TYC3"].astype(str)))
ts1 = Table([TYC, ts1["HIP"], ts1["RAdeg"], ts1["DEdeg"], ts1["VTmag"]], names=["TYC", "HIP", "alpha", "delta", "mag"])
print(ts1.info)

print(len(ts1))
for i in range(len(ts1)):
  stars.add_row(ts1[i])
print(stars.info)
stars.write("stars.fits", overwrite=True)

<Table length=1584>
 name  dtype  unit            description                class     n_bad
----- ------- ---- ---------------------------------- ------------ -----
  TYC   str12                                               Column     0
  HIP   int64                        Hipparcos number MaskedColumn   782
alpha float64  deg Right Asc, ICRS, at epoch=J1991.25       Column     0
delta float64  deg      Decl, ICRS, at epoch=J1991.25       Column     0
  mag float64  mag     Tycho-1 VT or Hp magnitude (4) MaskedColumn     0

1584
<Table length=329515>
 name  dtype  unit       description          class     n_bad 
----- ------- ---- ----------------------- ------------ ------
  TYC   str12               TYC Identifier       Column      0
  HIP   int64               HIP Identifier MaskedColumn 220578
alpha float64  deg Right Ascension (J2000)       Column      0
delta float64  deg     Declination (J2000)       Column      0
  mag float64  mag            VT Magnitude MaskedColumn      0
